In [72]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib import request
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import re
import sys
import json

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [73]:
# """
# 起動1
# java -jar C:\Users\mmi-lab\Documents\selenium\selenium-server-standalone-3.141.59.jar -role hub

# 起動2
# java -Dwebdriver.chrome.driver="C:\Users\mmi-lab\Documents\selenium\chromedriver.exe" -jar C:\Users\mmi-lab\Documents\selenium\selenium-server-standalone-3.141.59.jar -role node -hub http://192.168.1.224:4444

# """

# """
# 使えそう
# ジャンルからフレーズ・例文を探す
# https://gogakuru.com/english/phrase/genre/index.html

# """
print("aa")

aa


In [74]:
def is_declinable(token):
    pos = token.tag_.split("-")
    try:
        if pos[0] == "動詞" or pos[0] == "形容詞" or pos[2] == "サ変可能":
            # print(token.lemma_)
            return True
        else:
            False
    except:
        return False

In [75]:

def NER(t):
    doc = nlp(t)
    for ent in doc.ents:
        print(ent)

In [76]:
def depend_extract(t): 
    doc = nlp(t)
    for sent in doc.sents:
        for token in sent:
            # if is_declinable(token):
            if token.pos_ == "VERB":
                    # print()
                    # register_pred_count(pred_cound_dict, token.lemma_)
                for c in token.children:
                        # print(c)
                    for c2 in c.children:
                        if c2.dep_ == "case":
        
                            print( c.lemma_, c2, token.lemma_)
                            # register_pred_case_lite(pred_case_count_dict, token.lemma_, c2.text, c)
                            # print()

In [77]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [78]:
t = "インターネットは排出量が多い"
t = "マドリッドの原宿、表参道エリアを中心に人気専門店が続々登場するなど、今年一躍話題になったポップコーンですが、日本穀物検定協会が金曜ロードSHOW!とコラボレーションしたきょうは会社休みます。を12月に発売するそうです。"
# # t = "固有抽出表現は、文から日付や地名等の固有表現を抽出する処理です。"
# t = "ペトリコールは雨ですね"
# t  ="旅色というスペインの赤ワインや、プレーンズ・トレインズ・アンド・エリック〜ジャパン・ツアー 2014というマドリッドの白ワインはとても甘口なのでワインが苦手な方にもおすすめです。"
# t = "フランスの画家宇佐美貴史の油絵「牧場しぼり
# 」が、ボルドーで競売に掛けられ、予想額を大幅に上回る約７５億円で落札されました。"
# t = "腰が治りますよねー"
t = "私の住んでいる竹田城跡では、カミングアウトバラエティ秘密のケンミンSHOWがテレビ放送されていません。"

In [79]:
# depend_extract(t)

In [80]:
from spacy import displacy
displacy.render(nlp(t), style='dep', jupyter=True, options={'compact':True, 'distance': 90})

In [81]:
doc = nlp(t)
text = ""
for span in doc.noun_chunks:
    noun = span.text
    print(span)
    # for left in span.rights:
    #     print("\t", left)
    #     noun += left.text
    #     r = left
    #     for rr in r.rights:
    #     #    if rr.pos_
    #         print("\t\t", rr)
    #         r = rr
    #         noun += rr.text
            
    #         # noun -= rr.text
    #     print(noun)
    #     text += noun
    #     # リセット
    #     # noun = span.text
    # print()
    # print(text)

私
いる竹田城跡
カミングアウトバラエティ秘密
ケンミンSHOW


In [82]:
import ginza
def to_dependency_data(doc):
    words = []
    arcs = []
    bunsetu_head_list = ginza.bunsetu_head_list(doc)
    for i, chunk in enumerate(ginza.bunsetu_spans(doc)):
        words.append({
            'text': chunk.text, 'tag': chunk.label_
        })
        for token in chunk.lefts:
            arcs.append({
                'start': bunsetu_head_list.index(token.i),
                'end': i,
                'label': token.dep_,
                'dir': 'left'
            })
        for token in chunk.rights:
            arcs.append({
                'start': i,
                'end': bunsetu_head_list.index(token.i),
                'label': token.dep_,
                'dir': 'right'
            })
    return {'words': words, 'arcs': arcs}

displacy.render(
    to_dependency_data(nlp(t)),
    style='dep',
    jupyter=True,
    manual=True
)

In [83]:

t = "私の住んでいる竹田城跡では、カミングアウトバラエティ秘密のケンミンSHOWがテレビ放送されていません。"
bunsetu_ngram = []
n = 2
bunsetu = list( map( str, ginza.bunsetu_spans(doc) ) ) 
for i in range( len(bunsetu)-n+1 ):
    bunsetu_ngram.append( "".join(bunsetu[i:i+n]) )

In [84]:
bunsetu_ngram

['私の住んでいる',
 '住んでいる竹田城跡では、',
 '竹田城跡では、カミング',
 'カミングアウトバラエティ秘密の',
 'アウトバラエティ秘密のケンミンSHOWが',
 'ケンミンSHOWがテレビ放送されていません。']

In [85]:
import time
from selenium.webdriver.common.by import By
import spacy
import csv
import copy

In [86]:
class Seacher:
    def __init__(self) -> None:
        self.query = ""
        self.stop_word = "← ww ・ 「 」 #".split()
        self.nlp =  spacy.load('ja_ginza')

        # self.corpus_path = corpus_path
        # self.name = name

        self.tweet_all = set()
    
    def connect(self, server=None):
        chrome_options = webdriver.ChromeOptions()
        if server:
            command = "http://{0}:4444/wd/hub".format(server)
            self.driver = webdriver.Remote(
                command_executor="http://{0}:4444/wd/hub".format(server),
                options=chrome_options
            )
    
    def query2formated(self, query):
        # splited = query.split()
        return "\""+query+"\""
    
    def search(self, query):
        url = "https://twitter.com/search?f=live&q="

        self.driver.get( url+self.query2formated(query) )

        # 検索結果があるかチェック
        time.sleep(2)
        return self.is_exist_article()
            
    
    def is_exist_article(self):
        # print("start")
        html = self.driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        self.articles = soup.find_all("article")
        # print(self.articles)
        # print("come here")
        if self.articles==[]:
            return False
        else:
            return True
    
    


In [106]:
idf_path = "../../corpus/wiki/idf_wiki_v2.json"
with open(idf_path, "r") as f:
    idf_dict = json.load(f)

In [107]:
# idf_border = 11

def check_idf(doc):
    phrases = ginza.bunsetu_phrase_spans(doc)
    for phrase in phrases:
        key = phrase.lemma_
        if key in idf_dict:
            print(key, idf_dict[key])
        else:
            print("\tnot found : {0}".format(key))

In [108]:
def is_under_idf_border(doc, idf_border):
    phrases = ginza.bunsetu_phrase_spans(doc)
    for phrase in phrases:
        key = phrase.lemma_
        if key in idf_dict:
            if idf_dict[key] > idf_border:
                return False
        else:
            return False
    
    return True

In [109]:
def is_under_idf_border_morpheme(text, idf_border):
    for token in mecab_tokenize(text):
        key = token
        if key in idf_dict:
            if idf_dict[key] > idf_border:
                return False
        else:
            return False
    return True

In [110]:
t = "インターネットは排出量が多い"
t = "マドリッドの原宿、表参道エリアを中心に人気専門店が続々登場するなど、今年一躍話題になったポップコーンですが、日本穀物検定協会が金曜ロードSHOW!とコラボレーションしたきょうは会社休みます。を12月に発売するそうです。"
t = "固有抽出表現は、文から日付や地名等の固有表現を抽出する処理です。"
t = "ペトリコールは雨ですね"
# t  ="旅色というスペインの赤ワインや、プレーンズ・トレインズ・アンド・エリック〜ジャパン・ツアー 2014というマドリッドの白ワインはとても甘口なのでワインが苦手な方にもおすすめです。"
# t = "フランスの画家宇佐美貴史の油絵「牧場しぼり
# 」が、ボルドーで競売に掛けられ、予想額を大幅に上回る約７５億円で落札されました。"
t = "腰が治りますよねー"
# t = "夕食は二人で行った"
# t = "私の住んでいる竹田城跡では、カミングアウトバラエティ秘密のケンミンSHOWがテレビ放送されていません。"

In [111]:
border_10 = 14.62592894814336
border_1 = 9.88253708481772
print(clean_text(t))
is_under_idf_border_morpheme(clean_text(t), border_10)
# (nlp(clean_text(t)))

腰が治りますよねー


True

In [112]:
# def make_bunsetu_ngram(text, n=3):
#     bunsetu_ngram = []
#     bunsetu = list( map( str, ginza.bunsetu_spans(nlp(text)) ) ) 
#     for i in range( len(bunsetu)-n+1 ):
#         bunsetu_ngram.append( "".join(bunsetu[i:i+n]) )
#     return bunsetu_ngram

def make_bunsetu_ngram_suf(text, n=3):
    bunsetu_ngram = []
    doc = nlp(text)
    bunsetu = list(map(str, ginza.bunsetu_spans(doc))) 
    phrases = ginza.bunsetu_phrase_spans(doc)
    # print(bunsetu)
    # print(phrases)
    # for i in range( len(bunsetu)-n+1 ):
        # bunsetu_ngram.append( "".join(bunsetu[i:i+n]) )
    noun_chunk = [ ]
    for i in range(len(bunsetu)-n+1):
        # print(bunsetu[i],end=" " )
        bunsetu_ngram.append("".join(bunsetu[i:i+n-1])+phrases[i+n-1].orth_)

    # if bunsetu_ngram==[]:
    #     bunsetu_ngram = make_bunsetu_ngram_suf(text, n=2)
    return bunsetu_ngram

t = "私の住んでいる竹田城跡では、カミングアウトバラエティ秘密のケンミンSHOWがテレビ放送されていません。"
make_bunsetu_ngram_suf(t)

['私の住んでいる竹田城跡',
 '住んでいる竹田城跡では、カミング',
 '竹田城跡では、カミングアウトバラエティ秘密',
 'カミングアウトバラエティ秘密のケンミンSHOW',
 'アウトバラエティ秘密のケンミンSHOWがテレビ放送']

In [113]:
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

error = "Wrong information"
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y.append(1)
            else:
                y.append(0)

In [114]:
seacher = Seacher()
seacher.connect("192.168.1.224")

In [115]:
print("len:", len(y))
print("count({0}): {1}".format(error, y.count(1)))

len: 1386
count(Wrong information): 395


In [116]:
q = "赤ワインや、プレーンズ・トレインズ・アンド・エリック"
# seacher.search(q)

In [117]:
y_pred = []
border_10 = 14.62592894814336
for i, utt in enumerate(sys_utt):
    # print(utt)
    phrase_ngram = make_bunsetu_ngram_suf(utt, n=2)
    # 文節が2つしかない
    if phrase_ngram==[]:
        phrase_ngram = [utt]
    print(i, phrase_ngram)
    error = False

    # idf チェック
    if is_under_idf_border_morpheme( clean_text(utt), border_1):
        print("\t under idf, not search", utt)
        print()
        y_pred.append(0)
        continue

    for q in phrase_ngram:
        # ここでもチェック
        if is_under_idf_border_morpheme( clean_text(q), border_1):
            continue    
        if not seacher.search(q):
            print("not found", utt, q)
            error = True
            break
    # break
    # yのチェック
    if error:
        y_pred.append(1)
    else:
        y_pred.append(0)
    print()

0 ['お、そう', 'そうですか。\u3000', '\u3000えっ', 'えっと、四国', '四国は行か', '行かれたこと', 'ことあり']
	 under idf, not search お、そうですか。　えっと、四国は行かれたことありますか？

1 ['お遍路の逆回り', '逆回りは迷い', '迷いやすくて大変', '大変なぶん', 'ぶん、ご利益', 'ご利益が大きい']
	 under idf, not search お遍路の逆回りは迷いやすくて大変なぶん、ご利益が大きいらしいよ。

2 ['お兄さんに生まれる', '生まれると、頼りがい', '頼りがいのある', 'ある性格', '性格になる']
	 under idf, not search お兄さんに生まれると、頼りがいのある性格になるのかな。

3 ['元気ですかは元気']
not found 元気ですかは元気です 元気ですかは元気

4 ['元気ですかは元気']
not found 元気ですかは元気ですね 元気ですかは元気

5 ['もちろんですってのが元気', '元気ですかにいい']
not found もちろんですってのが元気ですかにいいですね 元気ですかにいい

6 ['もちろんですってのが元気', '元気ですかにいい']

7 ['私も好き']
	 under idf, not search 私も好きです

8 ['好きですよ']
	 under idf, not search 好きですよ

9 ['好きだを見']
	 under idf, not search 好きだを見ますよねー

10 ['好きだを連発']
	 under idf, not search 好きだを連発ですよねえ

11 ['あ、そう', 'そうですか。そう', '\u3000そうだ、芸能人', '芸能人に会っ', '会ったこと', 'ことあり']
	 under idf, not search あ、そうですか。　そうだ、芸能人に会ったことありますか？

12 ['人間関係を深めるため', '深めるためのあいさつ', 'あいさつとしてよく', 'よく用い']
	 under idf, not search 人間関係を深めるためのあいさつとしてよく用いられるよね。

13 ['新しい会社', '会社での人間関係'

In [118]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

confusion matrix = 
 [[823 168]
 [107 288]]
accuracy =  0.8015873015873016
precision =  0.631578947368421
recall =  0.7291139240506329
f1 score =  0.6768507638072855


- 旧バージョン

        confusion matrix = 
        [[519 454]
        [  2 374]]
        accuracy =  0.6619718309859155
        precision =  0.45169082125603865
        recall =  0.9946808510638298
        f1 score =  0.6212624584717609


- idf 搭載バージョン v1 border 10%

        confusion matrix = 
        [[724 249]
        [ 25 351]]
        accuracy =  0.7968865826538176
        precision =  0.585
        recall =  0.9335106382978723
        f1 score =  0.7192622950819672

- idf 搭載バージョン v2 border 10%

        confusion matrix = 
        [[756 217]
        [ 40 336]]
        accuracy =  0.809488510007413
        precision =  0.6075949367088608
        recall =  0.8936170212765957
        f1 score =  0.7233584499461788

### mecab version
- mecab idf version 1


- mecab idf version 2

        confusion matrix = 
        [[799 174]
        [101 275]]
        accuracy =  0.7961452928094885
        precision =  0.6124721603563474
        recall =  0.7313829787234043
        f1 score =  0.6666666666666667

In [119]:

# for x, p in zip(sys_utt, y):
#     if p==1:
#         print(x)

In [120]:
y.count(1)

395

In [121]:
def make_bunsetu_ngram_suf(text, n=3):
    bunsetu_ngram = []
    doc = nlp(text)
    bunsetu = list(map(str, ginza.bunsetu_spans(doc))) 
    phrases = ginza.bunsetu_phrase_spans(doc)
    # print(bunsetu)
    # print(phrases)
    # for i in range( len(bunsetu)-n+1 ):
        # bunsetu_ngram.append( "".join(bunsetu[i:i+n]) )
    noun_chunk = [ ]
    for i in range(len(bunsetu)-n+1):
        # print(bunsetu[i],end=" " )
        bunsetu_ngram.append("".join(bunsetu[i:i+n-1])+phrases[i+n-1].lemma_)

    if bunsetu_ngram==[]:
        bunsetu_ngram = make_bunsetu_ngram_suf(text, n=2)
    return bunsetu_ngram
tt = "腰も治りますよね～"
make_bunsetu_ngram_suf(tt)

['腰も治る']

In [122]:
data_path = "./wrong/"
data_name = "wrong_tw_res.pickle"
dataM = DataManager(data_path)

In [123]:
dataM.save_data(data_name, [y, y_pred])

success save : ./wrong/wrong_tw_res.pickle
